# Chapter 7 - Hypothesis and Inference

What will we do with all this statistics and probability theory? The science part of data
science frequently involves forming and testing hypotheses about our data and the
processes that generate it.

## Statistical Hypothesis Testing

Often, as data scientists, we’ll want to test whether a certain hypothesis is likely to be
true. For our purposes, hypotheses are assertions like “this coin is fair” or “data scien‐
tists prefer Python to R” or “people are more likely to navigate away from the page
without ever reading the content if we pop up an irritating interstitial advertisement
with a tiny, hard-to-find close button” that can be translated into statistics about data.
Under various assumptions, those statistics can be thought of as observations of random variables from known distributions, which allows us to make statements about
how likely those assumptions are to hold.

In the classical setup, we have a null hypothesis H0 that represents some default position, and some alternative hypothesis H1 that we’d like to compare it with. We use statistics to decide whether we can reject H0 as false or not. This will probably make
more sense with an example.

## Example: Flipping a Coin

Imagine we have a coin and we want to test whether it’s fair. We’ll make the assumption that the coin has some probability p of landing heads, and so our null hypothesis $H_0$: p = 0.5 (the coin is fair). We’ll test this against the alternative hypothesis $H_1$: p ≠ 0.5 (the coin isn't fair).

In particular, our test will involve flipping the coin some number n times and the random variable X is the number of heads in n-times coin flipping. Each coin flip is a Bernoulli trial, which means that X is a Binomial(n,p) random variable, which (as we saw in Chapter 6) we can approximate
using the normal distribution:

In [20]:
import math

In [21]:
def normal_approx_to_binomial(n,p):
    """Finds mu and sigma corresponding to a Binomial(n,p)."""
    mu = n*p
    sigma = math.sqrt(n*p*(1-p))
    return mu, sigma

Whenever a random variable follows a normal distribution, we can use normal_cdf
to figure out the probability that its realized value lies within (or outside) a particular
interval:

In [22]:
def normal_cdf(x, mu=0, sigma=1):
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

In [23]:
# the normal cdf _is_ the probability the variable is below a threshold
normal_prob_below = normal_cdf

# it's above the threshold if it's not below the threshold
def normal_prob_above(lo, mu=0, sigma=1):
    return 1-normal_cdf(lo, mu, sigma)

# it's between if it's less than hi, but not less than lo
def normal_prob_between(lo, hi, mu=0, sigma=1):
    return normal_cdf(hi,mu,sigma) - normal_cdf(lo,mu,sigma)

# it's outside if it's not between
def normal_prob_outside(lo, hi, mu=0, sigma=1):
    return 1-normal_prob_between(lo, hi, mu, sigma)

We can also do the reverse—find either the nontail region or the (symmetric) interval
around the mean that accounts for a certain level of likelihood. For example, if we
want to find an interval centered at the mean and containing 60% probability, then
we find the cutoffs where the upper and lower tails each contain 20% of the probability (leaving 60%):

In [24]:
def inverse_normal_cdf(p, mu=0, sigma=1, tolerance=0.00001):
    """Find approx. inverse using binary search."""
    
    # if not standard, compute standard and rescale
    if mu != 0 and sigma != 1:
        return mu+sigma*inverse_normal_cdf(p, tolerance=tolerance)
    
    low_z, low_p = -10.0, 0  # normal_cdf(-10) is (very close to) 0
    hi_z, hi_p = 10.0, 0     # normal_cdf(10) is (very close to) 1
    
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z)/2  # Consider the midpoint.
        mid_p = normal_cdf(mid_z) # and the cdf's value there
        
        if mid_p < p:
            # midpoint is still too low, search above it
            low_z, low_p = mid_z, mid_p
        elif mid_p > p:
            # midpoint is still too high, search below it
            hi_z, hi_p = mid_z, mid_p
        else:
            break
    
    return mid_z

In [25]:
def normal_upper_bound(probability, mu=0, sigma=1):
    """Return the z for which P(Z<=z) = probability."""
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability, mu=0, sigma=1):
    """Returns the z for which P(Z>=z) = probability."""
    return inverse_normal_cdf(1-probability, mu, sigma)

def normal_two_sided_bounds(probability, mu=0, sigma=1):
    """Returns the symmetric (about the mean) bounds that 
       contain the specified probability."""
    tail_probability = (1-probability)/2
    
    # upper bound should have tail_probability above it
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    
    # lower bound should have tail_probability below it
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    
    return lower_bound, upper_bound

In particular, let’s say that we choose to flip the coin n = 1000 times. If our hypothesis
of fairness is true, X (number of heads in n-times coin flipping) should be distributed approximately normally with mean 500 and standard deviation 15.8. Such result can be confirmed from normal approximation of binomial distribution:

In [29]:
mu_0, sigma_0 = normal_approx_to_binomial(1000, 0.5)
print('mu_0 =',mu_0, ', sigma_0 =',sigma_0)

mu_0 = 500.0 , sigma_0 = 15.811388300841896


In ordere to design statistical test, two things must be considered:

1. *Threshold of test*. Null hypothesis $H_0$ can be rejected when X is observed to be outside of certain thereshold.
2. *Power of test*. The test should be design such that the probability of fail to reject null hypothesis $H_0$, evethough it's false, is small.


To determine the threshold, we need to make a decision about *__significance—how willing we are to make a type 1
error (“false positive”), in which we reject $H_{0}$ even though it’s true__*. For reasons lost to
the annals of history, this willingness is often set at 5% or 1%. Let’s choose 5%.

Consider the test that rejects $H_{0}$ if X falls outside the bounds given by:

In [27]:
normal_two_sided_bounds(0.95, mu_0, sigma_0)

(469.01026640487555, 530.9897335951244)

*__Assuming p really equals 0.5 (i.e., $H_{0}$ is true), there is just a 5% chance we observe an
X that lies outside this interval, which is the exact significance we wanted__*. Said differently, if $H_{0}$ is true, then, approximately 19 times out of 20, this test will give the correct result.

*__The power of a test, which is the probability of not
making a type 2 error, in which we fail to reject $H_{0}$ even though it’s false__*. In order to
measure this, we have to specify what exactly $H_{0}$ being false means. (Knowing merely
that p is not 0.5 doesn’t give you a ton of information about the distribution of X.) In
particular, let’s check what happens if p is really 0.55, so that the coin is slightly biased
toward heads.

In that case, we can calculate the power of the test with:

In [30]:
# 95% bounds based on assumption p is 0.5
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

# Actual mu and sigma based on p = 0.55
mu_1, sigma_1 = normal_approx_to_binomial(1000, 0.55)

# A type 2 error means we fail to reject the null hypothesis
# which will happen when X is still in our original interval.
type_2_probability = normal_prob_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability

In [31]:
print(power)

0.8865480012953671


Imagine instead that our null hypothesis $H_0$ was that the coin is not biased toward heads,
or that $p \leq 0.5$ . In that case we want a *one-sided test* that rejects the null hypothesis
when X is much larger than 500 but not when X is smaller than 500. So a 5% significance test involves using normal_probability_below to find the cutoff below which 95% of the probability lies:

In [32]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)

In [33]:
print(hi)

526.0073585242053


In [35]:
type_2_prob = normal_prob_below(hi, mu_1, sigma_1)
power = 1 - type_2_prob 

In [36]:
print(power)

0.9363794803307174


This is a more powerful test, since it no longer rejects $H_0$ when X is below 469 (which
is very unlikely to happen if $H_1$ is true) and instead rejects $H_0$ when X is between 526
and 531 (which is somewhat likely to happen if $H_1$ is true).

An alternative way of thinking about the preceding test involves *p-values*. Instead of
choosing bounds based on some probability cutoff, *we compute the probability—
assuming $H_0$ is true—that we would see a value at least as extreme as the one we
actually observed*.

For our two-sided test of whether the coin is fair, we compute:

In [43]:
def two_sided_pvalue(x, mu=0, sigma=1):
    if x >= mu:
        # if x is greater than the mean, the tail is what's greater than x
        return 2*normal_prob_above(x, mu, sigma)
    else:
        # if x is less than the mean, the tail is what's less than x
        return 2*normal_prob_below(x, mu, sigma)

If we were to see 530 heads, we would compute:

In [44]:
two_sided_p_value(529.5, mu_0, sigma_0)

0.06207721579598835

Why did we use 529.5 instead of 530? This is what’s called a continuity correction. It reflects the fact that normal_probability_between(529.5, 530.5, mu_0, sigma_0) is a better estimate
of the probability of seeing 530 heads than normal_probability_between(530, 531, mu_0, sigma_0) is.

Correspondingly, normal_probability_above(529.5, mu_0,
sigma_0) is a better estimate of the probability of seeing at least
530 heads. 

One way to convince yourself that this is a sensible estimate is with a simulation:

In [42]:
import random

extreme_value_count = 0

for _ in range(100000):
    num_heads = sum(1 if random.random()<0.5 else 0 
                    for _ in range(1000)) # Count number of Heads in 1000 flips.
    if num_heads >= 530 or num_heads <= 470:
        extreme_value_count += 1          # Count how often the number of Heads is extreme.
        
print(extreme_value_count/100000)  

0.06123


Since the p-value is greater than our 5% significance, we don’t reject the null. If we
instead saw 532 heads, the p-value would be:

In [45]:
two_sided_pvalue(531.5, mu_0, sigma_0)

0.046345287837786575

which is smaller than the 5% significance, which means we would reject the null. It’s
the exact same test as before. It’s just a different way of approaching the statistics.

Similarly, we would have: 

In [46]:
upper_pvalue = normal_prob_above
lower_pvalue = normal_prob_below

For one-sided test, if we saw 525 heads we would compute:

In [47]:
upper_pvalue(524.5, mu_0, sigma_0)

0.06062885772582072

which means we wouldn’t reject the null. If we saw 527 heads, the computation would
be:

In [48]:
upper_pvalue(526.5, mu_0, sigma_0)

0.04686839508859242

and we would reject the null.

Make sure your data is roughly normally distributed before using
normal_probability_above to compute p-values. The annals of
bad data science are filled with examples of people opining that the
chance of some observed event occurring at random is one in a
million, when what they really mean is “the chance, assuming the
data is distributed normally,” which is pretty meaningless if the data
isn’t.

There are various statistical tests for normality, but even plotting
the data is a good start.

## Confidence Intervals

We’ve been testing hypotheses about the value of the heads probability p, which is a
parameter of the unknown “heads” distribution. When this is the case, a third
approach is to construct a confidence interval around the observed value of the
parameter.

For example, we can estimate the probability of the unfair coin by looking at the aver‐
age value of the Bernoulli variables corresponding to each flip—1 if heads, 0 if tails. If
we observe 525 heads out of 1,000 flips, then we estimate p equals 0.525.

How confident can we be about this estimate? Well, if we knew the exact value of p,
the central limit theorem (recall “The Central Limit Theorem” on page 78) tells us
that the average of those Bernoulli variables should be approximately normal, with
mean p and standard deviation:

    math.sqrt(p*(1-p)/1000)

Here we don’t know p, so instead we use our estimate:

In [51]:
import math 

p_hat = 525/1000
mu = p_hat
sigma = math.sqrt(p_hat*(1-p_hat)/1000)

print('mu=',mu,', sigma=',sigma)

mu= 0.525 , sigma= 0.015791611697353755


This is not entirely justified, but people seem to do it anyway. Using the normal
approximation, we conclude that we are “95% confident” that the following interval
contains the true parameter p:

In [52]:
normal_two_sided_bounds(0.95, mu, sigma)

(0.4940490278129096, 0.5559509721870904)

This is a statement about the interval, not about p. You should
understand it as the assertion that if you were to repeat the experiment many times, 95% of the time the “true” parameter (which is
the same every time) would lie within the observed confidence
interval (which might be different every time).

In particular, we do not conclude that the coin is unfair, since 0.5 falls within our con‐
fidence interval.

If instead we’d seen 540 heads, then we’d have:

In [55]:
p_hat = 540/1000
mu = p_hat
sigma = math.sqrt(p_hat*(1-p_hat)/1000)

print('mu=',mu,', sigma=',sigma)

mu= 0.54 , sigma= 0.015760710643876435


In [56]:
normal_two_sided_bounds(0.95, mu, sigma)

(0.5091095927295919, 0.5708904072704082)

Here, “fair coin” doesn’t lie in the confidence interval. (The “fair coin” hypothesis
doesn’t pass a test that you’d expect it to pass 95% of the time if it were true.)

## P-hacking

A procedure that erroneously rejects the null hypothesis only 5% of the time will—by
definition—5% of the time erroneously reject the null hypothesis:

In [60]:
import random

In [92]:
def run_exp():
    """Flip a fair coin 1000 times. True == Heads, False == Tail."""
    return [random.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment):
    """using the 5% significance levels"""
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

random.seed(0)
experiments = [run_exp() for _ in range(1000)]
num_rejections = len([experiment for experiment in experiments if reject_fairness(experiment)])

print(num_rejections)

46


What this means is that if you’re setting out to find “significant” results, you usually
can. Test enough hypotheses against your data set, and one of them will almost certainly appear significant. Remove the right outliers, and you can probably get your p
value below 0.05.

This is sometimes called P-hacking and is in some ways a consequence of the “infer‐
ence from p-values framework.”

*__If you want to do good science, you should determine your hypotheses before looking
at the data, you should clean your data without the hypotheses in mind, and you
should keep in mind that p-values are not substitutes for common sense__*. 

## Example: Running an A/B Test

One of your primary responsibilities at DataSciencester is experience optimization,
which is a euphemism for trying to get people to click on advertisements. One of
your advertisers has developed a new energy drink targeted at data scientists, and the
VP of Advertisements wants your help choosing between advertisement A (“tastes
great!”) and advertisement B (“less bias!”).

Being a scientist, you decide to run an experiment by randomly showing site visitors
one of the two advertisements and tracking how many people click on each one.

If 990 out of 1,000 A-viewers click their ad while only 10 out of 1,000 B-viewers click
their ad, you can be pretty confident that A is the better ad. But what if the differences
are not so stark? Here’s where you’d use statistical inference.

Let’s say that $N_A$ people see ad A, and that $n_A$ of them click it. We can think of each ad
view as a Bernoulli trial where $p_A$ is the probability that someone clicks ad A. Then (if
$N_A$ is large, which it is here) we know that $n_A / N_A$ is approximately a normal random
variable with mean $p_A$ and standard deviation $\sigma_A = \sqrt{p_A (1-p_A)/N_A}$.

Similarly, $n_B / N_B$ is approximately a normal random variable with mean $p_B$ and standard deviation $\sigma_B = \sqrt{p_B (1-p_B)/N_B}$:

In [93]:
def estimated_param(N,n):
    p = n/N
    sigma = math.sqrt(p*(1-p)/N)
    return p,sigma

If we assume those two normals are independent (which seems reasonable, since the
individual Bernoulli trials ought to be), then their difference should also be normal
with mean $p_B - p_A$ and standard deviation $\sqrt{\sigma_A^2 + \sigma_B^2}$. 

This is sort of cheating. The math only works out exactly like this if
you know the standard deviations. Here we’re estimating them
from the data, which means that we really should be using a tdistribution. But for large enough data sets, it’s close enough that it doesn’t make much of a difference.

This means we can test the null hypothesis that $p_A$ and $p_B$ are the same (that is, that
$p_B - p_A$ is zero), using the statistic:

In [94]:
import math

In [97]:
def a_b_statistics(N_A, n_A, N_B, n_B):
    p_A, sigma_A = estimated_param(N_A, n_A)
    p_B, sigma_B = estimated_param(N_B, n_B)
    return (p_B - p_A)/math.sqrt(sigma_A**2 + sigma_B**2)

which should approximately be a standard normal.

For example, if “tastes great” gets 200 clicks out of 1,000 views and “less bias” gets 180
clicks out of 1,000 views, the statistic equals:

In [98]:
z = a_b_statistics(1000, 200, 1000, 180)
print(z)

-1.1403464899034472


The probability of seeing such a large difference if the means were actually equal
would be:

In [99]:
two_sided_pvalue(z)

0.254141976542236

which is large enough that you can’t conclude there’s much of a difference. On the
other hand, if “less bias” only got 150 clicks, we’d have:

In [101]:
z = a_b_statistics(1000, 200, 1000, 150)
print(z)

-2.948839123097944


In [102]:
two_sided_pvalue(z)

0.003189699706216853

which means there’s only a 0.003 probability you’d see such a large difference if the
ads were equally effective.

## Bayesian Inference

The procedures we’ve looked at have involved making probability statements about
our tests: “there’s only a 3% chance you’d observe such an extreme statistic if our null
hypothesis were true.”

An alternative approach to inference involves treating the unknown parameters
themselves as random variables. The analyst (that’s you) starts with a prior distribution for the parameters and then uses the observed data and Bayes’s Theorem to get
an updated posterior distribution for the parameters. *__Rather than making probability
judgments about the tests, you make probability judgments about the parameters
themselves__*.

For example, when the unknown parameter is a probability (as in our coin-flipping
example), we often use a prior from the Beta distribution, which puts all its probabil‐
ity between 0 and 1:

In [103]:
def B(alpha, beta):
    """a normalizing constant so that the total probability is 1"""
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x, alpha, beta):
    if x < 0 or x > 1: # no weight outside of [0, 1]
        return 0
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)

Generally speaking, this distribution centers its weight at:

    alpha / (alpha + beta)
    
and the larger alpha and beta are, the “tighter” the distribution is.

For example, if alpha and beta are both 1, it’s just the uniform distribution (centered
at 0.5, very dispersed). If alpha is much larger than beta, most of the weight is near 1.
And if alpha is much smaller than beta, most of the weight is near zero. Figure 7-1
shows several different Beta distributions.

So let’s say we assume a prior distribution on p. Maybe we don’t want to take a stand
on whether the coin is fair, and we choose alpha and beta to both equal 1. Or maybe
we have a strong belief that it lands heads 55% of the time, and we choose alpha
equals 55, beta equals 45.

Then we flip our coin a bunch of times and see h heads and t tails. Bayes’s Theorem
(and some mathematics that’s too tedious for us to go through here) tells us that the
posterior distribution for p is again a Beta distribution but with parameters alpha +
h and beta + t.

It is no coincidence that the posterior distribution was again a Beta
distribution. The number of heads is given by a Binomial distribu‐
tion, and the Beta is the conjugate prior to the Binomial distribu‐
tion. This means that whenever you update a Beta prior using
observations from the corresponding binomial, you will get back a
Beta posterior.

![](beta_dist.jpg)

Let’s say you flip the coin 10 times and see only 3 heads.
If you started with the uniform prior (in some sense refusing to take a stand about
the coin’s fairness), your posterior distribution would be a Beta(4, 8), centered around
0.33. Since you considered all probabilities equally likely, your best guess is some‐
thing pretty close to the observed probability.

If you started with a Beta(20, 20) (expressing the belief that the coin was roughly
fair), your posterior distribution would be a Beta(23, 27), centered around 0.46, indi‐
cating a revised belief that maybe the coin is slightly biased toward tails.

And if you started with a Beta(30, 10) (expressing a belief that the coin was biased to
flip 75% heads), your posterior distribution would be a Beta(33, 17), centered around
0.66. In that case you’d still believe in a heads bias, but less strongly than you did ini‐
tially. These three different posteriors are plotted in Figure 7-2.

![](post_dist.jpg)

If you flipped the coin more and more times, the prior would matter less and less
until eventually you’d have (nearly) the same posterior distribution no matter which
prior you started with.

For example, no matter how biased you initially thought the coin was, it would be
hard to maintain that belief after seeing 1,000 heads out of 2,000 flips (unless you are
a lunatic who picks something like a Beta(1000000,1) prior).

What’s interesting is that this allows us to make probability statements about hypotheses: “Based on the prior and the observed data, there is only a 5% likelihood the
coin’s heads probability is between 49% and 51%.” This is philosophically very different from a statement like “if the coin were fair we would expect to observe data so
extreme only 5% of the time.”

Using Bayesian inference to test hypotheses is considered somewhat controversial—
in part because its mathematics can get somewhat complicated, and in part because of
the subjective nature of choosing a prior. We won’t use it any further in this book, but
it’s good to know about.